In [1]:
import numpy as np
import pandas as pd
import time
import heapq
import json
import psycopg2
import psycopg2.extras
import warnings
import sys
import uuid
import cvxpy as cp
import datetime
import argparse
from pandasql import sqldf

INF = float('inf')
total_time = time.time()

if sys.version_info[0:2] != (3, 6):
    warnings.warn('Please use Python3.6', UserWarning)

numLimit = 5
threadLimit = 4
queryID = "Jvg5KZdwpToJ8hWGScumoy"
debug = 1


In [2]:
def ReportStatus(msg, flag, queryID, query_version, output=None):
    """
    Print message and update status in biz_model.biz_fir_query_parameter_definition.
    """
    sql = "update fll_t_dw.biz_fir_query_parameter_definition set python_info_data='{0}', success_flag='{1}', update_time='{2}', python_result_json='{3}', version={5} where id='{4}'".format(msg, flag, datetime.datetime.now(), output, queryID, query_version)
    print("============================================================================================================================")
    print("Reporting issue:", msg)
    conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute(sql)
    sql = "update fll_t_dw.biz_fir_third_parameter_definition set python_info_data='{0}', success_flag='{1}', update_time='{2}', python_result_json='{3}', version={5} where id='{4}'".format(msg, flag, datetime.datetime.now(), output, queryID, query_version)
    cur.execute(sql)
    conn.commit()
    conn.close()

def ConnectDatabase(queryID):
    """
    Load parameters in JSON from biz_model.biz_fir_query_parameter_definition and load data from biz_model.biz_ads_fir_pkg_data.
    """
    try:
        print('Parameters reading...')
        sqlParameter = "select query_id, query_based_version, query_version, python_json from fll_t_dw.biz_fir_third_parameter_definition where id='{0}'".format(queryID)
        conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
        paramInput = pd.read_sql(sqlParameter, conn)
        if paramInput.shape[0] == 0:
            raise Exception("No Valid Query Request is Found!")
        elif paramInput.shape[0] > 1:
            raise Exception("More than One Valid Query Requests are Found!")
        param = json.loads(paramInput['python_json'][0])
        data_query_id = paramInput['query_id'][0]
        query_based_verion = paramInput['query_based_version'][0]
        query_version = paramInput['query_version'][0]
        print("base version: {0}\ncurrent version: {1}\nobjective: {2}".format(query_based_verion, query_version, param["objective"]))
    except Exception as e:
        print("Loading Parameters from GreenPlum Failed!\n", e)
        exit(1)
    try:
        print('Data loading...')
        # sqlInput = """
        #     select billing_status_fz as billing, unit_id_fz as unit_id, product, fleet_year_fz as fleet_year, contract_cust_id as customer, contract_num as contract_num, \
        #     contract_lease_type as contract, cost, nbv, age_x_ceu as weighted_age, query_id, ceu_fz as ceu, teu_fz as teu, rent as rent, rml_x_ceu as rml
        #     from biz_model.biz_ads_fir_pkg_data WHERE query_id='{0}'
        # """.format(queryID) 
        sqlInput = \
        """
        select billing_status_fz, unit_id_fz, product, fleet_year_fz, contract_cust_id, contract_num,
        contract_lease_type, cost, nbv, age_x_ceu, ceu_fz, teu_fz, rent, rml_x_ceu_c, cust_country, per_diem_rate_current
        from fll_t_dw.biz_ads_fir_pkg_data
        where unit_id_fz in
        (select unit_id
        from fll_t_dw.biz_fir_asset_package
        where query_version = {0}
        and query_id = '{1}')
        """.format(query_based_verion, data_query_id)
        data = pd.read_sql(sqlInput, conn)
        print("Executing pandas SQL...")
        pysqldf = lambda q: sqldf(q, globals())
        q = \
        """
        select billing_status_fz as billing, unit_id_fz as unit_id, p1.product, fleet_year_fz as fleet_year, contract_cust_id as customer, p1.contract_num,
        contract_lease_type as contract, cost, nbv, age_x_ceu as weighted_age, ceu_fz as ceu, teu_fz as teu, rent as rent, rml_x_ceu_c as rml, cust_country
        from data p1
        inner join 
        (select contract_num, product
        from (select contract_num, product, count(*) num from data group by 1, 2) p1 
        where num >= {0}) p2
        on p1.contract_num=p2.contract_num and p1.product=p2.product
        """.format(param["numContractProductLimit"])
        data = pysqldf(q)
        if data.shape[0] == 0:
            raise Exception("No Data Available in GreenPlum!")
        print('Input data shape:', data.shape)
        conn.close()
    except Exception as e:
        print(e)
        ReportStatus("Loading Data from GreenPlum Failed!", 'F', queryID, query_version)
        exit(1)

    return param, data, query_version

def OutputPackage(data, result, queryID, query_version):
    """
    Output final package to biz_model.biz_fir_asset_package.
    """
    sqlOutput = "insert into fll_t_dw.biz_fir_asset_package (unit_id, query_id, id, is_void, version, query_version) values %s"
    try:
        conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
        conn.autocommit = True
        cur = conn.cursor()
        print('Writing data...')
        values_list = []
        for i in range(len(result)):
            if result[i]:
                values_list.append((data['unit_id'][i], queryID, uuid.uuid1().hex, 0, 0, query_version))
        psycopg2.extras.execute_values(cur, sqlOutput, values_list)
        conn.commit()
        conn.close()
    except Exception as e:
        print(e) 
        ReportStatus("Writing data to GreenPlum Failed!", 'F', queryID, query_version)
        exit(1)


In [5]:
param, data, query_version = ConnectDatabase(queryID)

Parameters reading...
base version: 2
current version: 5
objective: {'objective': 'remainingLeaseTenor', 'type': None, 'maxOrMin': '0', 'value': 2, 'basis': None}
Data loading...


In [4]:
try:
    timeLimit = param['timeLimit'] if param['timeLimit'] > 0 else 600
    if debug:
        print('model time limit:', timeLimit)
    numContractProductLimit = param['numContractProductLimit']
    NbvCost = param['prefer']['nbvorCost']
    maxOrMin = param['prefer']['maxOrMin']

    # objective
    objectiveTarget = param['objective']['objective']
    objectiveType = param['objective']['type']
    objectiveMaxOrMin = param['objective']['maxOrMin']
    objectiveValue = param['objective']['value']
    objectiveBasis = param['objective']['basis']

    fleetAgeLowBound = [-INF for _ in range(numLimit)]
    fleetAgeUpBound = [INF for _ in range(numLimit)]
    fleetAgeLimit = [None for _ in range(numLimit)]
    fleetAgeGeq = [None for _ in range(numLimit)]
    weightedAgeLowBound = [-INF for _ in range(numLimit)]
    weightedAgeUpBound = [INF for _ in range(numLimit)]
    weightedAgeLimit = [None for _ in range(numLimit)]
    weightedAgeGeq = [None for _ in range(numLimit)]
    lesseeType = [None for _ in range(numLimit)]
    lesseeLimit = [None for _ in range(numLimit)]
    lesseeGeq = [None for _ in range(numLimit)]
    productType = [None for _ in range(numLimit)]
    productLimit = [None for _ in range(numLimit)]
    productGeq = [None for _ in range(numLimit)]
    contractType = [None for _ in range(numLimit)]
    contractLimit = [None for _ in range(numLimit)]
    contractGeq = [None for _ in range(numLimit)]
    statusType = [None for _ in range(numLimit)]
    statusLimit = [None for _ in range(numLimit)]
    statusGeq = [None for _ in range(numLimit)]
    rmlLowBound = [-INF for _ in range(numLimit)]
    rmlUpBound = [INF for _ in range(numLimit)]
    rmlGeq = [None for _ in range(numLimit)]
    rmlLimit = [None for _ in range(numLimit)]
    countryName = [None for _ in range(numLimit)]
    countryGeq = [None for _ in range(numLimit)]
    countryLimit = [None for _ in range(numLimit)]

    minTotalNbv = param['totalNBVFrom']
    maxTotalNbv = param['totalNBVTo']

    minTotalCost = param['totalCostFrom']
    maxTotalCost = param['totalCostTo']

    minTotalRent = param['totalRentFrom']

    lesseeOthers = param['lessee']['others']['lessee']
    lesseeOthersLimit = param['lessee']['others']['percent'] / 100

    topLesseeLimit = [
        param['lessee']['topLessee']['top1']['percent'] / 100,
        param['lessee']['topLessee']['top2']['percent'] / 100,
        param['lessee']['topLessee']['top3']['percent'] / 100]
    topLesseeGeq = [
        param['lessee']['topLessee']['top1']['symbol'],
        param['lessee']['topLessee']['top2']['symbol'],
        param['lessee']['topLessee']['top3']['symbol']]

    fleetAgeAvgLimit = param['containersAge']['average']['averageContainersAge']
    fleetAgeAvgGeq = param['containersAge']['average']['symbol']
    fleetAgeBasis = param['containersAge']['basis']
    for i in range(len(param['containersAge']['list'])):
        fleetAgeLowBound[i] = param['containersAge']['list'][i]['containersAgeFrom']
        fleetAgeUpBound[i] = param['containersAge']['list'][i]['containersAgeTo']
        fleetAgeLimit[i] = param['containersAge']['list'][i]['percent'] / 100
        fleetAgeGeq[i] = param['containersAge']['list'][i]['symbol']

    weightedAgeAvgLimit = param['weightedAge']['average']['averageWeighedAge']
    weightedAgeAvgGeq = param['weightedAge']['average']['symbol']
    weightedAgeBasis = param['weightedAge']['basis']
    for i in range(len(param['weightedAge']['list'])):
        weightedAgeLowBound[i] = param['weightedAge']['list'][i]['weightedAgeFrom']
        weightedAgeUpBound[i] = param['weightedAge']['list'][i]['weightedAgeTo']
        weightedAgeLimit[i] = param['weightedAge']['list'][i]['percent'] / 100
        weightedAgeGeq[i] = param['weightedAge']['list'][i]['symbol']

    lesseeBasis = param['lessee']['basis']
    for i in range(len(param['lessee']['list'])):
        lesseeType[i] = param['lessee']['list'][i]['lessee']
        lesseeLimit[i] = param['lessee']['list'][i]['percent'] / 100
        lesseeGeq[i] = param['lessee']['list'][i]['symbol']

    statusBasis = param['status']['basis']
    for i in range(len(param['status']['list'])):
        statusType[i] = param['status']['list'][i]['statusType']
        statusLimit[i] = param['status']['list'][i]['percent'] / 100
        statusGeq[i] = param['status']['list'][i]['symbol']

    productBasis = param['product']['basis']
    for i in range(len(param['product']['list'])):
        productType[i] = param['product']['list'][i]['productType']
        productLimit[i] = param['product']['list'][i]['percent'] / 100
        productGeq[i] = param['product']['list'][i]['symbol']

    contractBasis = param['contractType']['basis']
    for i in range(len(param['contractType']['list'])):
        contractType[i] = param['contractType']['list'][i]['contractType']
        contractLimit[i] = param['contractType']['list'][i]['percent'] / 100
        contractGeq[i] = param['contractType']['list'][i]['symbol']

    rmlBasis = param['rml']['basis']
    for i in range(len(param['rml']['list'])):
        rmlLowBound[i] = param['rml']['list'][i]['rmlFrom']
        # rmlUpBound[i] = param['rml']['list'][i]['rmlTo']
        rmlGeq[i] = param['rml']['list'][i]['symbol']
        rmlLimit[i] = param['rml']['list'][i]['percent'] / 100

    countryBasis = param['country']['basis']
    for i in range(len(param['country']['list'])):
        countryName[i] = param['country']['list'][i]['country']
        countryGeq[i] = param['country']['list'][i]['symbol']
        countryLimit[i] = param['country']['list'][i]['percent'] / 100
except Exception as e:
    print(e)
    msg = 'Parsing Paramters Failed! ' + str(e)
    ReportStatus(msg, 'F', queryID, query_version)
    exit(1)
print("==============================================================")
print('Data processing...')
try:
    # Billing Status
    data['OnHireStatus'] = data['billing'].apply(lambda x: 1 if x=='ON' else 0)
    data['OffHireStatus'] = data['billing'].apply(lambda x: 1 if x=='OF' else 0)
    data['NoneStatus'] = data['billing'].apply(lambda x: 1 if (x!='ON' and x!='OF') else 0)
    # One hot all lessees
    for l in data['customer'].value_counts().index:
        data[l] = data['customer'].apply(lambda x: 1 if x==l else 0)
    # One hot all contract number
    for c in data['contract_num'].value_counts().index:
        data[c] = data['contract_num'].apply(lambda x: 1 if x==c else 0)
    # One hot all product type
    for p in data['product'].value_counts().index:
        data[p] = data['product'].apply(lambda x: 1 if x==p else 0)
    for i in range(numLimit):
        # Container Age
        if fleetAgeLimit[i]:
            data['FleetAge{0}'.format(i)] = data['fleet_year'].apply(lambda x: 1 if fleetAgeLowBound[i]<=x<=fleetAgeUpBound[i] else 0)
        # Weighted Age
        if weightedAgeLimit[i]:
            data['WeightedAge{0}'.format(i)] = data['weighted_age'].apply(lambda x: 1 if weightedAgeLowBound[i]<=x<=weightedAgeUpBound[i] else 0)
        # Product Type
        if productLimit[i]:
            data['ProductType{0}'.format(i)] = data['product'].apply(lambda x: 1 if x in productType[i] else 0)
        # Contract Type
        if contractLimit[i]:
            data['ContractType{0}'.format(i)] = data['contract'].apply(lambda x: 1 if x in contractType[i] else 0)
        # RML
        if rmlLimit[i]:
            data['RML{0}'.format(i)] = data['rml'].apply(lambda x: 1 if rmlLowBound[i]<=x<=rmlUpBound[i] else 0)
        # Country
        if countryLimit[i]:
            data['Country{0}'.format(i)] = data['cust_country'].apply(lambda x: 1 if x in countryName[i] else 0)
    # convert data to numpy
    nbv = data['nbv'].to_numpy()
    cost = data['cost'].to_numpy()
    ceu = data['ceu'].to_numpy()
    teu = data['teu'].to_numpy()
    rent = data['rent'].to_numpy()
    fleetAgeAvg = data['fleet_year'].to_numpy()
    weightedAgeAvg = data['weighted_age'].to_numpy()
    onHireStatus = data['OnHireStatus'].to_numpy()
    offHireStatus = data['OffHireStatus'].to_numpy()
    noneHireStatus = data['NoneStatus'].to_numpy()
    lesseeOneHot = {l: data[l].to_numpy() for l in data['customer'].value_counts().index}
    contractNumOneHot = [data[c].to_numpy() for c in data['contract_num'].value_counts().index]
    productTypeOneHot = [data[p].to_numpy() for p in data['product'].value_counts().index]
    fleetAge = []
    weightedAge = []
    product = []
    contract = []
    rml = []
    country = []

    for i in range(numLimit):
        fleetAge.append(data['FleetAge{0}'.format(i)].to_numpy() if fleetAgeLimit[i] else None)
        weightedAge.append(data['WeightedAge{0}'.format(i)].to_numpy() if weightedAgeLimit[i] else None)
        product.append(data['ProductType{0}'.format(i)].to_numpy() if productLimit[i] else None)
        contract.append(data['ContractType{0}'.format(i)].to_numpy() if contractLimit[i] else None)
        rml.append(data['RML{0}'.format(i)].to_numpy() if rmlLimit[i] else None)
        country.append(data['Country{0}'.format(i)].to_numpy() if countryLimit[i] else None)
    basis = {}
    basis['nbv'] = nbv
    basis['ceu'] = ceu
    basis['teu'] = teu
    basis['cost'] = cost
    hireStatus = {}
    hireStatus['ON'] = onHireStatus
    hireStatus['OF'] = offHireStatus
    hireStatus['None'] = noneHireStatus 
except Exception as e:
    print(e)
    msg = 'Processing Data Failed!' + str(e)
    ReportStatus(msg, 'F', queryID, query_version)
    exit(1)


model time limit: 100
Data processing...
'billing'
Reporting issue: Processing Data Failed!'billing'


SyntaxError: syntax error at or near "billing"
LINE 1: ...ion set python_info_data='Processing Data Failed!'billing'',...
                                                             ^


In [ ]:
# TODO: this is just for development
objectiveTarget = 'containerAge'
objectiveMaxOrMin = 1

def BuildModel():
    print("==============================================================")
    print('Model preparing...')
    start_time = time.time()
 
    x = cp.Variable(shape=data.shape[0], boolean=True)
    y = cp.Variable(shape=data.shape[0])
    t = cp.Variable(shape=1)
    z = cp.Variable(shape=1, boolean=True)
    constraints = []

    # objective function 
    if objectiveTarget == 'containerAge':
        if objectiveMaxOrMin:
            objective = cp.Maximize(cp.sum(fleetAgeAvg@x) / cp.sum(x))
            print('++++++++++++++++++')
            # objective = cp.Maximize(1)
        else:
            objective = cp.Minimize(cp.sum(cp.multiply(y, fleetAgeAvg)))

    # nbv
    if maxTotalNbv:
        constraints.append(cp.sum(cp.multiply(x, nbv)) <= maxTotalNbv)
        if debug:
            print('\tNBV <= {0}'.format(maxTotalNbv))
    if minTotalNbv:
        constraints.append(cp.sum(cp.multiply(x, nbv)) >= minTotalNbv)
        if debug:
            print('\tNBV >= {0}'.format(minTotalNbv))
    # cost
    if maxTotalCost:
        constraints.append(cp.sum(cp.multiply(x, cost)) <= maxTotalCost)
        if debug:
            print('\tCost <= {0}'.format(maxTotalCost))
    if minTotalCost:
        constraints.append(cp.sum(cp.multiply(x, cost)) >= minTotalCost)
        if debug:
            print('\tCost >= {0}'.format(minTotalCost))
    # rent
    if minTotalRent:
        constraints.append(cp.sum(cp.multiply(x, rent)) >= minTotalRent)
        if debug:
            print('\tRent >= {0}'.format(minTotalRent))
    # container age
    if fleetAgeAvgLimit:
        if debug:
            print('\tAverage Age {0} {1}'.format('>=' if fleetAgeAvgGeq else '<=', fleetAgeAvgLimit))
        if fleetAgeAvgGeq:
            constraints.append(cp.sum(cp.multiply(x, fleetAgeAvg)) >= fleetAgeAvgLimit * cp.sum(x))
        else:
            constraints.append(cp.sum(cp.multiply(x, fleetAgeAvg)) <= fleetAgeAvgLimit * cp.sum(x))
    if fleetAgeBasis:
        for i in range(numLimit):
            if fleetAgeLimit[i]:
                if debug:
                    print('\tContainer Age {0}-{1} {2} {3}'.format(fleetAgeLowBound[i], fleetAgeUpBound[i], '>=' if fleetAgeGeq[i] else '<=', fleetAgeLimit[i]))
                if fleetAgeGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, fleetAge[i] * basis[fleetAgeBasis])) >=
                        fleetAgeLimit[i] * cp.sum(cp.multiply(x, basis[fleetAgeBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, fleetAge[i] * basis[fleetAgeBasis])) <=
                        fleetAgeLimit[i] * cp.sum(cp.multiply(x, basis[fleetAgeBasis])))
    # weighted age
    if weightedAgeAvgLimit:
        if debug:
            print('\tAverage Weighted Age {0} {1}'.format('>=' if weightedAgeAvgGeq else '<=', weightedAgeAvgLimit))
        if weightedAgeAvgGeq:
            constraints.append(cp.sum(cp.multiply(x, weightedAgeAvg)) >=
                weightedAgeAvgLimit * cp.sum(cp.multiply(x, ceu)))
        else:
            constraints.append(cp.sum(cp.multiply(x, weightedAgeAvg)) <=
                weightedAgeAvgLimit * cp.sum(cp.multiply(x, ceu)))
    if weightedAgeBasis:
        for i in range(numLimit):
            if weightedAgeLimit[i]:
                if debug:
                    print('\tWeighted Age {0}-{1} {2} {3}'.format(weightedAgeLowBound[i], weightedAgeUpBound[i], '>=' if weightedAgeGeq[i] else '<=', weightedAgeLimit[i]))
                if weightedAgeGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, weightedAge[i] * basis[weightedAgeBasis])) >=
                        weightedAgeLimit[i] * cp.sum(cp.multiply(x, basis[weightedAgeBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, weightedAge[i] * basis[weightedAgeBasis])) <=
                        weightedAgeLimit[i] * cp.sum(cp.multiply(x, basis[weightedAgeBasis])))
    # lessee
    if lesseeBasis:
        for i in range(numLimit):
            if lesseeLimit[i]:
                if lesseeType[i] not in lesseeOneHot:
                    print('\tlessee {0} does not exist'.format(lesseeType[i]))
                    continue
                    # lesseeOneHot[lesseeType[i]] = np.zeros(data.shape[0])
                if debug:
                    print('\tLessee {0} {1} {2}'.format(lesseeType[i], '>=' if lesseeGeq[i] else '<=', lesseeLimit[i]))
                if lesseeGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, lesseeOneHot[lesseeType[i]] * basis[lesseeBasis])) >=
                        lesseeLimit[i] * cp.sum(cp.multiply(x, basis[lesseeBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, lesseeOneHot[lesseeType[i]] * basis[lesseeBasis])) <=
                        lesseeLimit[i] * cp.sum(cp.multiply(x, basis[lesseeBasis])))
        maxTop = 0
        # top lessee -- top lessee does not support >=
        for i in range(3):
            if topLesseeLimit[i]:
                maxTop = i+1
                if debug:
                    print('\tTop{0} lessee <= {1}'.format(maxTop, topLesseeLimit[i]))
                constraints.append(cp.sum_largest(
                    cp.hstack([cp.sum(cp.multiply(x, lesseeOneHot[l] * basis[lesseeBasis])) for l in lesseeOneHot]), i+1) <=
                        topLesseeLimit[i] * cp.sum(cp.multiply(x, basis[lesseeBasis])))
        # others
        if lesseeOthersLimit:
            if lesseeOthers:
                if debug:
                    print('\tOther lessees <= {0}'.format(lesseeOthersLimit))
                # add constraints according to user input
                constraints.append(cp.sum_largest(
                    cp.hstack([cp.sum(cp.multiply(x, lesseeOneHot[l] * basis[lesseeBasis])) for l in lesseeOthers]), 1) <=
                        lesseeOthersLimit * cp.sum(cp.multiply(x, basis[lesseeBasis])))
    # status
    if statusBasis:
        for i in range(numLimit):
            if statusType[i]:
                if debug:
                    print('\tStatus {0} {1} {2}'.format(statusType[i], '>=' if statusGeq[i] else '<=', statusLimit[i]))
                if statusGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, hireStatus[statusType[i]] * basis[statusBasis])) >=
                        statusLimit[i] * cp.sum(cp.multiply(x, basis[statusBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, hireStatus[statusType[i]] * basis[statusBasis])) <=
                        statusLimit[i] * cp.sum(cp.multiply(x, basis[statusBasis])))
    # product
    if productBasis:
        for i in range(numLimit):
            if productLimit[i]:
                if debug:
                    print('\tProduct {0} {1} {2}'.format(productType[i], '>=' if productGeq[i] else '<=', productLimit[i]))
                if productGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, product[i] * basis[productBasis])) >=
                        productLimit[i] * cp.sum(cp.multiply(x, basis[productBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, product[i] * basis[productBasis])) <=
                        productLimit[i] * cp.sum(cp.multiply(x, basis[productBasis])))
    # contract type
    if contractBasis:
        for i in range(numLimit):
            if contractLimit[i]:
                if debug:
                    print('\tContract {0} {1} {2}'.format(contractType[i], '>=' if contractGeq[i] else '<=', contractLimit[i]))
                if contractGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, contract[i] * basis[contractBasis])) >=
                        contractLimit[i] * cp.sum(cp.multiply(x, basis[contractBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, contract[i] * basis[contractBasis])) <=
                        contractLimit[i] * cp.sum(cp.multiply(x, basis[contractBasis])))
    # rml
    if rmlBasis:
        for i in range(numLimit):
            if rmlLimit[i]:
                if debug:
                    print('\tRML {0}-inf {1} {2}'.format(rmlLowBound[i], '>=' if rmlGeq[i] else '<=', rmlLimit[i]))
                if rmlGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, rml[i] * basis[rmlBasis])) >=
                        rmlLimit[i] * cp.sum(cp.multiply(x, basis[rmlBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, rml[i] * basis[rmlBasis])) <=
                        rmlLimit[i] * cp.sum(cp.multiply(x, basis[rmlBasis])))

    # customer country
    if countryBasis:
        for i in range(numLimit):
            if countryLimit[i]:
                if debug:
                    print('\tLessee country {0} {1} {2}'.format(countryName[i], '>=' if countryGeq[i] else '<=', countryLimit[i]))
                if countryGeq[i]:
                    constraints.append(cp.sum(cp.multiply(x, country[i] * basis[countryBasis])) >= 
                        countryLimit[i] * cp.sum(cp.multiply(x, basis[countryBasis])))
                else:
                    constraints.append(cp.sum(cp.multiply(x, country[i] * basis[countryBasis])) <= 
                        countryLimit[i] * cp.sum(cp.multiply(x, basis[countryBasis])))

    # set number of contract_num & product_type limit
    if numContractProductLimit:
        contractProductType = [c*p for c in contractNumOneHot for p in productTypeOneHot if sum(c*p) > 0]
        delta = cp.Variable(shape=len(contractProductType), boolean=True)
        if debug:
            print('\t# of contract-product >= {0}'.format(numContractProductLimit))
        # constraints.append(cp.sum_smallest(
        #     cp.hstack([cp.sum(cp.multiply(x, c*p)) for c in contractNumOneHot for p in productTypeOneHot if sum(c*p) > 0]), 1) >= numContractProductLimit)
        for i in range(len(contractProductType)):
            constraints.append(cp.sum(cp.multiply(x, contractProductType[i])) >= numContractProductLimit * delta[i])
            constraints.append(cp.sum(cp.multiply(x, contractProductType[i])) <= 9999999 * delta[i])
    prob = cp.Problem(objective, constraints)
    print('Time Cost', time.time() - start_time)


    return prob, x

def SolveModel(prob, timeLimit, threadLimit):
    start_time = time.time()
    print("==============================================================")
    print('Model solving...')
    # solve model
    prob.solve(solver=cp.CBC, verbose=0, qcp=True, maximumSeconds=timeLimit, numberThreads=threadLimit)
    print("==============================================================")
    print("status:", prob.status)
    print("==============================================================")
    print('Time Cost', time.time() - start_time)
    return prob


In [ ]:
prob, x = BuildModel()
x.value = np.ones(shape=len(data))
start_time = time.time()
print("==============================================================")
print('Model solving...')
# solve model
prob.solve(solver='CBC', warm_start=True, verbose=0, qcp=1, max_iters=10)
print("==============================================================")
print("status:", prob.status)
print("==============================================================")
print('Time Cost', time.time() - start_time)

NameError: name 'BuildModel' is not defined

In [ ]:
def ValidResult(result):
    reportJson = {}
    passed = True
    print("==============================================================")
    # NBV
    resultNbv = sum(result*nbv)
    print("nbv: {0}".format(round(resultNbv, 4)))
    reportJson['nbv'] = resultNbv
    if maxTotalNbv:
        if (resultNbv - maxTotalNbv) > 0.1:
            passed = False
            print('\tmax failed')
    if minTotalNbv:
        if (minTotalNbv - resultNbv) > 0.1: 
            passed = False
            print('\tmin failed')
    # COST
    resultCost = sum(result*cost)
    print("cost: {0}".format(round(resultCost, 4)))
    reportJson['cost'] = resultCost
    if maxTotalCost:
        if (resultCost - maxTotalCost) > 0.1:
            passed = False
            print('\tmax failed')
    if minTotalCost:
        if (minTotalCost - resultCost) > 0.1:
            passed = False
            print('\tmin failed')
    # RENT
    resultRent = sum(result*rent)
    print("rent: {0}".format(round(resultRent, 4)))
    reportJson['rent'] = resultRent
    if minTotalRent:
        if (minTotalCost - resultRent) > 0.1:
            passed = False
            print('\tmin failed')

    # Fleet Age
    if fleetAgeAvgLimit:
        resultFleetAgeAvg = sum(result*fleetAgeAvg)/sum(result)
        print('container average age is {0}'.format(round(resultFleetAgeAvg, 4)))
        reportJson['fleetAverageAge'] = resultFleetAgeAvg
        if fleetAgeAvgGeq:
            if resultFleetAgeAvg < fleetAgeAvgLimit:
                passed = False
                print('\t>= failed')
        else:
            if resultFleetAgeAvg > fleetAgeAvgLimit:
                passed = False
                print('\t<= failed')

    if fleetAgeBasis:
        l = []
        for i in range(numLimit):
            if fleetAgeLimit[i]:
                resultFleetAge = sum(result*fleetAge[i]*basis[fleetAgeBasis])/sum(result*basis[fleetAgeBasis])
                print("container age from {0} to {1} is {2}".format(fleetAgeLowBound[i], fleetAgeUpBound[i], round(resultFleetAge, 4)))
                tmp = {}
                tmp['from'] = fleetAgeLowBound[i]
                tmp['to'] = fleetAgeUpBound[i]
                tmp['percent'] = resultFleetAge * 100
                l.append(tmp)
                if fleetAgeGeq[i]:
                    if resultFleetAge < fleetAgeLimit[i]:
                        passed = False
                        print('\t>= failed')
                else:
                    if resultFleetAge > fleetAgeLimit[i]:
                        passed = False
                        print('\t<= failed')
        reportJson['fleetAge'] = l
    
    # Weighted Age
    if weightedAgeAvgLimit:
        resultWeightedAgeAvg = sum(result*weightedAgeAvg)/sum(result*ceu)
        print('weighted average age is {0}'.format(round(resultWeightedAgeAvg, 4)))
        reportJson['weightedAverageAge'] = resultWeightedAgeAvg
        if weightedAgeAvgGeq:
            if resultWeightedAgeAvg < weightedAgeAvgLimit:
                print('\t>= failed')
                passed = False
        else:
            if resultWeightedAgeAvg > weightedAgeAvgLimit:
                print('\t<= failed')
                passed = False

    if weightedAgeBasis:
        l = []
        for i in range(numLimit):
            if weightedAgeLimit[i]:
                resultWeightedAge = sum(result*weightedAge[i]*basis[weightedAgeBasis])/sum(result*basis[weightedAgeBasis])
                print("weighted age from {0} to {1} is {2}".format(weightedAgeLowBound[i], weightedAgeUpBound[i], round(resultWeightedAge, 4)))
                tmp = {}
                tmp['from'] = weightedAgeLowBound[i]
                tmp['to'] = weightedAgeUpBound[i]
                tmp['percent'] = resultWeightedAge * 100
                l.append(tmp)
                if weightedAgeGeq[i]:
                    if resultWeightedAge < weightedAgeLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultWeightedAge > weightedAgeLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['weightedAge'] = l
    
    # Lessee
    if lesseeBasis:
        l = []
        for i in range(numLimit):
            if lesseeLimit[i]:
                if lesseeType[i] not in lesseeOneHot:
                    print('lessee {0} does not exist'.format(lesseeType[i]))
                    continue
                resultLessee = sum(result*lesseeOneHot[lesseeType[i]]*basis[lesseeBasis])/sum(result*basis[lesseeBasis])
                print("lessee {0} is {1}".format(lesseeType[i], round(resultLessee, 4)))
                tmp = {}
                tmp['lessee'] = lesseeType[i]
                tmp['percent'] = resultLessee * 100
                l.append(tmp)
                if lesseeGeq[i]:
                    if resultLessee < lesseeLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultLessee > lesseeLimit[i]:
                        print('\t<= failed')
                        passed = False

        top4Lessee = heapq.nlargest(4, [(lesseeName, sum(result*lesseeOneHot[lesseeName]*basis[lesseeBasis])) for lesseeName in data['customer'].value_counts().index], key=lambda x:x[1])
        resultTop3Lessee = [
            sum(i[1] for i in top4Lessee[:1]) / sum(result*basis[lesseeBasis]),
            sum(i[1] for i in top4Lessee[:2]) / sum(result*basis[lesseeBasis]),
            sum(i[1] for i in top4Lessee[:3]) / sum(result*basis[lesseeBasis])
        ]
        for i in range(3):
            if topLesseeLimit[i]:
                print('top {0} {1} is {2}'.format(i+1, [i[0] for i in top4Lessee[:i+1]], round(resultTop3Lessee[i], 4)))
                tmp = {}
                tmp['top'] = i+1
                tmp['lessee'] = [i[0] for i in top4Lessee[:i+1]]
                tmp['percent'] = resultTop3Lessee[i] * 100
                l.append(tmp)
                if topLesseeGeq[i]:
                    if resultTop3Lessee[i] < topLesseeLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultTop3Lessee[i] > topLesseeLimit[i]:
                        print('\t<= failed')
                        passed = False
        # other lessee
        if lesseeOthersLimit:
            tmp = {}
            if lesseeOthers:
                print('Other lessees via list')
                maxOtherLessee = max([sum(result*lesseeOneHot[l]*basis[lesseeBasis]) for l in lesseeOthers]) / sum(result*basis[lesseeBasis])
                print('\ttop others is {0}'.format(round(maxOtherLessee, 4)))
                tmp['percentOthers'] = maxOtherLessee * 100
                if maxOtherLessee > lesseeOthersLimit:
                    print('\t\tfailed')
                    passed = False
            l.append(tmp)
        reportJson['lessee'] = l    

    # Status Type
    if statusBasis:
        l = []
        for i in range(numLimit):
            if statusType[i]:
                resultStatus = sum(result*hireStatus[statusType[i]] *basis[statusBasis])/sum(result*basis[statusBasis])
                print('Hire {0} is {1}'.format(statusType[i], round(resultStatus, 4)))
                tmp = {}
                tmp['status'] = statusType[i]
                tmp['percent'] = resultStatus * 100
                l.append(tmp)
                if statusGeq[i]:
                    if resultStatus < statusLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultStatus > statusLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['status'] = l
    
    # Product Type
    if productBasis:
        l = []
        for i in range(numLimit):
            if productLimit[i]:
                resultProduct = sum(result*product[i]*basis[productBasis])/sum(result*basis[productBasis])
                print("product {0} is {1}".format(productType[i], round(resultProduct, 4)))
                tmp = {}
                tmp['productType'] = productType[i]
                tmp['percent'] = resultProduct * 100
                l.append(tmp)
                if productGeq[i]:
                    if resultProduct < productLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultProduct > productLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['product'] = l

    # Contract Type
    if contractBasis:
        l = []
        for i in range(numLimit):
            if contractLimit[i]:
                resultContract = sum(result*contract[i]*basis[contractBasis])/sum(result*basis[contractBasis])
                print("contract type {0} is {1}".format(contractType[i], round(resultContract, 4))) 
                tmp = {}
                tmp['contractType'] = contractType[i]
                tmp['percent'] = resultContract * 100
                l.append(tmp)
                if contractGeq[i]:
                    if resultContract < contractLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultContract > contractLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['contract'] = l

    # Lessee Country
    if countryBasis:
        l = []
        for i in range(numLimit):
            if countryLimit[i]:
                resultCountry = sum(result*country[i]*basis[countryBasis])/sum(result*basis[countryBasis])
                print("Lessee Country {0} is {1}".format(countryName[i], round(resultCountry, 4)))
                tmp = {}
                tmp['lesseeCountryName'] = countryName[i]
                tmp['percent'] = resultCountry * 100
                l.append(tmp)
                if countryGeq[i]:
                    if resultCountry < countryLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultCountry > countryLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['lesseeCountry'] = l
    
    # RML
    if rmlBasis:
        l = []
        for i in range(numLimit):
            if rmlLimit[i]:
                resultRML = sum(result*rml[i]*basis[rmlBasis])/sum(result*basis[rmlBasis])
                print("rml from {0} to {1} is {2}".format(rmlLowBound[i], rmlUpBound[i], round(resultRML, 4)))
                tmp = {}
                tmp['from'] = rmlLowBound[i]
                tmp['to'] = rmlUpBound[i]
                tmp['percent'] = resultRML * 100
                l.append(tmp)
                if rmlGeq[i]:
                    if resultRML < rmlLimit[i]:
                        print('\t>= failed')
                        passed = False
                else:
                    if resultRML > rmlLimit[i]:
                        print('\t<= failed')
                        passed = False
        reportJson['rml'] = l
        
    # Contract-Product 
    if numContractProductLimit:
        contractProductTypeResult = [result*c*p for c in contractNumOneHot for p in productTypeOneHot if sum(result*c*p) > 0]
        minNumContractProduct = min([sum(i) for i in contractProductTypeResult])
        print('Minimum unit number of ProductNumber-ProductType is {0}'.format(minNumContractProduct))
        # print([sum(i) for i in contractProductTypeResult])
        reportJson['minNumContractProduct'] = int(minNumContractProduct)
        if minNumContractProduct < numContractProductLimit:
            print('\t failed')
            passed = False

    if passed:
        print('Algorithm Succeeded!!!!!!!!!!!!!!!!')
    return passed, json.dumps(reportJson)


In [ ]:

if prob.status == 'infeasible':
    ReportStatus('Problem Proven Infeasible! Please Modify Constaints.', 'I', queryID, query_version)
else:

    result = x.value
    print('Result is Valid:', len(set(result)) == 2)
    result = np.where(abs(result-1) < 1e-3, 1, 0) # x == 1
    print(int(sum(result)), '/', len(result), 'containers are selected.')

    if int(sum(result)) == 0:
        ReportStatus('Constraints Cannot Be fulfilled! Please Modify Constaints.', 'I', queryID, query_version)
    else:   
        passed, outputJson = ValidResult(result)
        OutputPackage(data, result, queryID, query_version)
        if passed:
            ReportStatus('Algorithm Succeeded!', 'O', queryID, query_version, outputJson)
        else:
            ReportStatus('Constraints Cannot Be fulfilled! Please Modify Constaints Or Increase Running Timelimit.', 'N', queryID, query_version, outputJson)

